# Calculate pO2 using T_bar, S_bar, O2_bar

In [1]:
import metabolic as mi
import Phi_derivative as Pd
import xarray as xr
import numpy as np
import os
import utils
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
var1 = 'O2' # mmol/m3
var2 = 'TEMP' # deg C
var3 = 'SALT' # PSS
var = [var1,var2,var3]

### DPLE hindcast of T,S,O2

In [3]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_monthly.nc'
ds_hind = xr.open_dataset(filename)
ds_hind

<xarray.Dataset>
Dimensions:     (z_t: 60, nlat: 384, nlon: 320, time: 840, d2: 2)
Coordinates:
  * z_t         (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT        (nlat, nlon) float64 ...
    TLONG       (nlat, nlon) float64 ...
    ULAT        (nlat, nlon) float64 ...
    ULONG       (nlat, nlon) float64 ...
  * time        (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon, d2
Data variables:
    O2          (time, z_t, nlat, nlon) float32 ...
    TEMP        (time, z_t, nlat, nlon) float32 ...
    SALT        (time, z_t, nlat, nlon) float32 ...
    time_bound  (time, d2) object ...
    KMT         (nlat, nlon) float64 ...
    TAREA       (nlat, nlon) float64 ...
    dz          (z_t) float32 ...

### calculate climatological-mean T or O2

In [4]:
nmon = ds_hind.dims['time']
aa = xr.DataArray(data=np.ones(nmon),coords=dict(time=('time',ds_hind.coords['time'].values)),dims='time')

In [6]:
O2_mean = ds_hind['O2'].mean(dim='time')
T_mean = ds_hind['TEMP'].mean(dim='time')
S_mean = ds_hind['SALT'].mean(dim='time')
#ds_hind['O2_mean'] = aa*O2_mean
#ds_hind['TEMP_mean'] = aa*T_mean
#ds_hind['SALT_mean'] = aa*S_mean
ds_hind['O2_mean'] = O2_mean
ds_hind['TEMP_mean'] = T_mean
ds_hind['SALT_mean'] = S_mean

### calculate pO2 based on O2, T, S, depth

In [7]:
depth = ds_hind['z_t']*ds_hind['O2_mean']/ds_hind['O2_mean']/1e2 # m

In [8]:
ds_hind['depth'] = depth.transpose("z_t","nlat","nlon")

In [9]:
ds_hind['depth']

<xarray.DataArray 'depth' (z_t: 60, nlat: 384, nlon: 320)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [ 5.0000005,  5.       ,  5.       , ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [15.       , 15.       , 15.       , ...,        nan,
                nan,        nan],
...
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT     (nlat, nlon) float64 ...
    TLONG    (nlat, nlon) float64 ...
    ULAT     (nlat, nlon) float64 ...
    ULONG    (nlat, nlon) float64 ...
Dimensions without coordinates: nlat, nlon

### save pO2 and O2_mean

In [10]:
dsf = xr.Dataset()
dsf['O2_mean'] = ds_hind['O2_mean']
dsf['T_mean'] = ds_hind['TEMP_mean']
dsf['S_mean'] = ds_hind['SALT_mean']
dsf['3ddepth'] = ds_hind['depth']

In [11]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:  (z_t: 60, nlat: 384, nlon: 320)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
    TLAT     (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    TLONG    (nlat, nlon) float64 320.6 321.7 322.8 323.9 ... 318.9 319.4 319.8
    ULAT     (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.42 72.41 72.41
    ULONG    (nlat, nlon) float64 321.1 322.3 323.4 324.5 ... 319.2 319.6 320.0
Dimensions without coordinates: nlat, nlon
Data variables:
    O2_mean  (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    T_mean   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    S_mean   (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    3ddepth  (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan

In [12]:
%%time
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_O2_T_S_mean_dimension_reduced_1948-2017_monthly.nc', mode='w')

CPU times: user 69.4 ms, sys: 33.3 ms, total: 103 ms
Wall time: 394 ms
